In [ ]:
import os.path
import numpy as np
from astropy.io import fits
from astropy.table import Table

def load_coadds(platefile, zbestfile=None, run1d=None):
    '''
    Document ...
    '''
    #- Load spPlate data
    fx = fits.open(platefile, memmap=False)
    header   = fx[0].header
    c0       = header['COEFF0']
    c1       = header['COEFF1']
    nwave    = header['NAXIS1']
    nfiber   = header['NAXIS2']
    wave     = (10**(c0 + c1*np.arange(nwave))).astype(np.float32)
    flux     = fx[0].data
    ivar     = fx[1].data
    and_mask = fx[2].data
    or_mask  = fx[3].data
    wavedisp = fx[4].data
    sky      = fx[6].data
    fx.close()

    if run1d is None:
        run1d = header['RUN2D']  #- default run1d == run2d
    
    #- Get best fit model from zbest file
    if zbestfile is None:
        zbestfile = platefile.replace('spPlate', '{}/spZbest'.format(run1d))

    model = fits.getdata(zbestfile, 2)

    coadds = list()
    for i in range(nfiber):
        sp = Table()
        sp['WAVE']     = wave               #- repeat !
        sp['FLUX']     = flux[i]
        sp['IVAR']     = ivar[i]
        sp['AND_MASK'] = and_mask[i]
        sp['OR_MASK']  = or_mask[i]
        sp['WAVEDISP'] = wavedisp[i]
        sp['SKY']      = sky[i]
        sp['MODEL']    = model[i]
        sp.meta = header
        
        #- TODO: Add units, comments to each column
        
        coadds.append(sp)
        
    return coadds

def load_frame(framefile, cframefile=None, flatfile=None):
    """
    Document ...
    """
    if cframefile is None:
        cframefile = framefile.replace('spFrame', 'spCFrame')
        if cframefile.endswith('.gz'):
            cframefile = cframefile[:-3]
    import os.path
    if os.path.exists(framefile)==False:
       exit()
    #- Load framefile and get original dimensions
    eflux = fits.getdata(framefile, 0)
    nfiber, npix = eflux.shape
    if os.path.exists(cframefile)==False:
     exit()        
    #- Load spCFrame file; trim arrays back to original size
    fx = fits.open(cframefile, memmap=False)
    header = fx[0].header
    flux = fx[0].data[:, 0:npix]
    ivar = fx[1].data[:, 0:npix]
    mask = fx[2].data[:, 0:npix]
    wave = (10**fx[3].data[:, 0:npix]).astype(np.float32)
    wavedisp  = fx[4].data[:, 0:npix]
    sky    = fx[6].data[:, 0:npix]
    x      = fx[7].data[:, 0:npix]
    superflat = fx[8].data[:, 0:npix]
    
    #- Load fiberflat spFlat[0]
    if flatfile is None:
        flatfile = header['FLATFILE'].replace('sdR', 'spFlat')
        flatfile = flatfile.replace('.fit', '.fits.gz')
        filedir, basename = os.path.split(os.path.abspath(cframefile))
        flatfile = os.path.join(filedir, flatfile)

    if os.path.exists(flatfile)==False:
      exit()
    fiberflat = fits.getdata(flatfile, 0)
    
    #- Calculate calibration vector: flux = electrons * calib
    electrons = eflux * fiberflat * superflat
    ii = np.where(electrons != 0.0)
    calib = np.zeros(flux.shape)
    calib[ii] = flux[ii] / electrons[ii]
            
    fx.close()
    
    #- Assemble spectra tables
    spectra = list()
    for i in range(nfiber):
        sp = Table()
        sp['WAVE'] = wave[i]
        sp['FLUX'] = flux[i]
        sp['IVAR'] = ivar[i]
        sp['MASK'] = mask[i]
        sp['WAVEDISP'] = wavedisp[i]
        sp['SKY'] = sky[i]
        sp['X'] = x[i]
        sp['CALIB'] = calib[i].astype(np.float32)
        sp.meta = header
        
        #- TODO: Add units, comments to each column
        
        spectra.append(sp)
        
    return spectra


In [1]:
"""
Create an HDF5 file from BOSS data

TODO:
  - include comments in meta/attrs
  - platelist quantities
"""

from __future__ import division, print_function
import sys, os
import numpy as np
from astropy.io import fits
from astropy.table import Table
import h5boss.io
import time

def serial_convert(platefile,hdf5output):
    platefile=platefile[0]
    hdf5output=str(hdf5output)
    filedir = os.path.split(os.path.abspath(platefile))[0]
    hdr = fits.getheader(platefile) 
    plate = hdr['PLATEID']
    mjd = hdr['MJD']
    tstart=time.time()
    #--- Plugmap ---
    print('plugmap')
    plugmap = Table.read(platefile, 5)
    dataname = '{}/{}/plugmap'.format(plate, mjd)
    plugmap.write(hdf5output, path=dataname, append=True)

    #--- zbest ---
    print('zbest')
    run1d = hdr['RUN2D']  #- default run1d == run2d
    zbestfile = platefile.replace('spPlate', '{}/spZbest'.format(run1d))
    zbest = Table.read(zbestfile, 1)
    dataname = '{}/{}/zbest'.format(plate, mjd)
    zbest.write(hdf5output, path=dataname, append=True)
    nfiber = len(zbest)

    #--- zall (skip) ---
    pass

    #--- zline ---
    print('zline')
    zlinefile = zbestfile.replace('spZbest-', 'spZline-')
    zline = Table.read(zlinefile, 1)
    dataname = '{}/{}/zline'.format(plate, mjd)
    zline.write(hdf5output, path=dataname, append=True)

    #--- photometric matches ---
    print('photo')
    photomatchfile = platefile.replace('spPlate-', 'photoMatchPlate-')
    photomatch = Table.read(photomatchfile, 1)
    photomatch['FIBERID'] = np.arange(1, nfiber+1, dtype=np.int16)
    dataname = '{}/{}/photo/match'.format(plate, mjd)
    photomatch.write(hdf5output, path=dataname, append=True)

    photoposfile = platefile.replace('spPlate-', 'photoPosPlate-')
    photopos = Table.read(photoposfile, 1)
    photopos['FIBERID'] = np.arange(1, nfiber+1, dtype=np.int16)
    dataname = '{}/{}/photo/matchpos'.format(plate, mjd)
    photopos.write(hdf5output, path=dataname, append=True)

    photofluxfile = platefile.replace('spPlate-', 'photoPlate-')
    photoflux = Table.read(photofluxfile, 1)
    photoflux['FIBERID'] = np.arange(1, nfiber+1, dtype=np.int16)
    dataname = '{}/{}/photo/matchflux'.format(plate, mjd)
    photoflux.write(hdf5output, path=dataname, append=True)

    #--- Coadd ---
    print('loading coadds')
    coadds = h5boss.io.load_coadds(platefile)

    print('writing coadds')
    for i, cx in enumerate(coadds):
        dataname = '{}/{}/{}/coadd'.format(plate, mjd, i+1)
        cx.write(hdf5output, path=dataname, append=True)

    #--- Individual exposures ---
    #- Parse spPlancomb to get exposures that were used
    print('parsing planfile')
    planfile = platefile.replace('spPlate-', 'spPlancomb-').replace('.fits', '.par')
    framefiles = list()
    for line in open(planfile):
        if line.startswith('SPEXP '):
            tmp = line.split()
            tmp = [x+'.gz' for x in tmp[7:-1]]
            framefiles.extend(tmp)

    print('individual exposures')
    for filename in framefiles:
        print(filename)
        frame = h5boss.io.load_frame(filedir+'/'+filename)
        if ('spFrame-b1' in filename) or ('spFrame-r1' in filename):
            offset = 0
        elif ('spFrame-b2' in filename) or ('spFrame-r2' in filename):
            offset = 500
        else:
            print('huh?', filename)
            sys.exit(1)

        for i, fx in enumerate(frame):
            br = fx.meta['CAMERAS'][0]
            expid = fx.meta['EXPOSURE']
            fiber = offset+i+1
            dataname = '{}/{}/{}/exposures/{}/{}'.format(plate, mjd, fiber, expid, br)
            fx.write(hdf5output, path=dataname, append=True)


    tend=time.time()-tstart
    print ('time',tend)  


ImportError: No module named 'h5boss'

# Rewrite the boss2hdf5 converter 

In [1]:
#sections in serial convert
from __future__ import division, print_function
import sys, os
import numpy as np
from astropy.io import fits
from astropy.table import Table
import time
datapath = "/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/v5_7_0/"
def listfiles():
     ldir=os.listdir(datapath)
     lldir=[fn for fn in ldir if fn.isdigit()]
     return lldir
plateslist=listfiles()
platepath_for_current_rank = datapath+plateslist[0]
def findseed(x):
     fitsfiles = [os.path.join(root, name)
       for root, dirs, files in os.walk(x)
       for name in files
        if name.startswith("spPlate") and name.endswith(".fits")]
     return fitsfiles
fitspath_name_for_current_rank = findseed(platepath_for_current_rank)
fitspath_name_for_current_rank

['/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/v5_7_0/5290/spPlate-5290-55862.fits']

In [2]:
#sections in load coadd
platefile=fitspath_name_for_current_rank[0]
filedir = os.path.split(os.path.abspath(platefile))[0]
hdr = fits.getheader(platefile) 
plate = hdr['PLATEID']
mjd = hdr['MJD']
#- Load spPlate data
fx = fits.open(platefile, memmap=False)
header   = fx[0].header
c0       = header['COEFF0']
c1       = header['COEFF1']
nwave    = header['NAXIS1']
nfiber   = header['NAXIS2']
wave     = (10**(c0 + c1*np.arange(nwave))).astype(np.float32)
flux     = fx[0].data
ivar     = fx[1].data
and_mask = fx[2].data
or_mask  = fx[3].data
wavedisp = fx[4].data
sky      = fx[6].data
fx.close()
run1d = header['RUN2D']  #- default run1d == run2d
    
#- Get best fit model from zbest file
zbestfile = platefile.replace('spPlate', '{}/spZbest'.format(run1d))
model = fits.getdata(zbestfile, 2)
coadds=(wave,flux,ivar,and_mask,or_mask,wavedisp,sky,model,header)
hdf5output="h5boss_v2_15.h5"
dat=("wave","flux","ivar","and_mask","or_mask","wavedisp","sky","model")
#list(header.cards)

In [3]:
#copy each wavelength dataset into the hdf5 file
import h5py
outx=h5py.File(hdf5output,'w')
for i in range(0,8):
    id = '{}/{}/{}'.format(plate, mjd, dat[i])
    dx=outx.create_dataset(id,data=coadds[i])
    temptb=Table()
    temptb.meta=coadds[8]
    #temptb.write(hdf5output,id,append=True)
    #dx.attrs.__setitem__(dat[i], coadds[8])


In [4]:
#--- Individual exposures ---
#- Parse spPlancomb to get exposures that were used
print('parsing planfile')
planfile = platefile.replace('spPlate-', 'spPlancomb-').replace('.fits', '.par')
framefiles = list()
for line in open(planfile):
    if line.startswith('SPEXP '):
        tmp = line.split()
        tmp = [x+'.gz' for x in tmp[7:-1]]
        framefiles.extend(tmp)

print('individual exposures')


parsing planfile
individual exposures


In [5]:
#sections in load exposures
def load_frame(framefile, cframefile=None, flatfile=None):
    """
    Document ...
    """
    if cframefile is None:
        cframefile = framefile.replace('spFrame', 'spCFrame')
        if cframefile.endswith('.gz'):
            cframefile = cframefile[:-3]
    import os.path
    if os.path.exists(framefile)==False:
        exit()
    #- Load framefile and get original dimensions
    eflux = fits.getdata(framefile, 0)
    nfiber, npix = eflux.shape
    if os.path.exists(cframefile)==False:
        exit()        
    #- Load spCFrame file; trim arrays back to original size
    fx = fits.open(cframefile, memmap=False)
    header = fx[0].header   # this means the exposureid is same for flux, ivar, etc
    expid=header['EXPOSURE']
    flux = fx[0].data[:, 0:npix]
    ivar = fx[1].data[:, 0:npix]
    mask = fx[2].data[:, 0:npix]
    wave = (10**fx[3].data[:, 0:npix]).astype(np.float32)
    wavedisp  = fx[4].data[:, 0:npix]
    sky    = fx[6].data[:, 0:npix]
    x      = fx[7].data[:, 0:npix]
    superflat = fx[8].data[:, 0:npix]
    
    #- Load fiberflat spFlat[0]
    if flatfile is None:
        flatfile = header['FLATFILE'].replace('sdR', 'spFlat')
        flatfile = flatfile.replace('.fit', '.fits.gz')
        filedir, basename = os.path.split(os.path.abspath(cframefile))
        flatfile = os.path.join(filedir, flatfile)

    if os.path.exists(flatfile)==False:
        exit()
    fiberflat = fits.getdata(flatfile, 0)
    
    #- Calculate calibration vector: flux = electrons * calib
    electrons = eflux * fiberflat * superflat
    ii = np.where(electrons != 0.0)
    calib = np.zeros(flux.shape)
    calib[ii] = flux[ii] / electrons[ii]
            
    fx.close()
    exposure=(wave,flux,ivar,mask,wavedisp,sky,x,calib.astype(np.float32), header,expid)    
    return exposure

In [6]:
expdat=("wave","flux","ivar","mask","wavedisp","sky","x","calib")
def dump2h5(frame1,frame2,expid,hdf5output,br):
    import h5py
    print ('dump:',expid)

    for i in range(0,len(expdat)):
        try:
            outx=h5py.File(hdf5output,'w')
        except Exception as e:
            print ("file open error")
            pass
        id = '{}/{}/exposures/{}/{}/{}'.format(plate, mjd, expid,br,expdat[i])
        try:
            dset=np.append(frame1[i],frame2[i])
            print ("id:%s"%id)
            print ("shape:",dset.shape)
        except Exception as e:
            print ("append error")
            pass
        try:
            dx=outx.create_dataset(id,data=dset)
        except Exception as e:
            print ('error in frame dump')
            pass
        #outx.flush()
        outx.close()    
def single_dump2h5(frame1,expid,hdf5output,br):
    import h5py
    print ("single dump:",expid)

    for i in range(0,len(expdat)):
        try:
            outx=h5py.File(hdf5output,'w')
        except Exception as e:
            print ("file open error")
            pass
        id = '{}/{}/exposures/{}/{}/{}'.format(plate, mjd, expid,br,expdat[i])
        dset=frame1[i]
        try:
            dx=outx.create_dataset(id,data=dset)
        except Exception as e:
            print ('error in frame dump')
            pass
        outx.flush()
        outx.close()        

In [7]:
# initialize the empty arrays, 
# b: wave, flux, ivar, mask, wavedisp, sky, x, calib, header
# r: wave, flux, ivar, mask, wavedisp, sky, x, calib, header
b1=0
r1=0
print (len(framefiles))
frameb1=list()
framer1=list()
frameb2=list()
framer2=list()
filename=framefiles[0]
if ('spFrame-b1' in filename):
    frame=load_frame(filedir+'/'+filename)
#for filename in framefiles:
#    offset = 0  #fiber 0-499
#    if ('spFrame-b1' in filename):
#        try:
#         frame=load_frame(filedir+'/'+filename)
#         frameb1.append(frame)
#         print (filename, frame[9])
#        except Exception as e:
#            print ("File not found")
#            pass
    #if ('spFrame-r1' in filename):
    #    try:
    #     frame=load_frame(filedir+'/'+filename)
    #     framer1.append(frame)
    #     print (filename, frame[9])
    #    except Exception as e:
    #        print ("File not found")
    #        pass
     

#combine b1 and b2


40


In [10]:
frame[0].shape

(500, 4112)

In [ ]:
for filename in framefiles:
    offset = 500 #fiber 500-999
    if ('spFrame-b2' in filename):
        try:
         frame=load_frame(filedir+'/'+filename)
         frameb2.append(frame)
         print (filename, frame[9])
        except Exception as e:
            print ("File not found")
            pass                
    if ('spFrame-r2' in filename):
        try:
         frame=load_frame(filedir+'/'+filename)
         framer2.append(frame)
         print (filename, frame[9])
        except Exception as e:
            print ("File not found")
            pass            

print ("frameb1:%d"%(len(frameb1)))        
print ("frameb2:%d"%(len(frameb2)))        
print ("framer1:%d"%(len(framer1)))        
print ("framer2:%d"%(len(framer2)))   

In [2]:
framefiles

NameError: name 'framefiles' is not defined

In [ ]:
for i in range(0,len(frameb1)):
    expidb1=frameb1[i][9]
    hit=0
    for j in range(0,len(frameb2)):
        expidb2=frameb2[j][9]
        if expidb1==expidb2:
            #combine frameb1 and frameb2
            dump2h5(frameb1[i],frameb2[j],expidb1,hdf5output,'b')
            frameb2.remove(frameb2[j])
            hit=1
            break
    if hit==0:
        single_dump2h5(frameb1[i],expidb1,hdf5output,'b')
for i in range(0,len(frameb2)):
    expidb2=frameb2[i][9]
    single_dump2h5(frameb2[i],expidb2,hdf5output,'b')

#combine r1 and r2
for i in range(0,len(framer1)):
    expidr1=framer1[i][9]
    hit=0
    for j in range(0,len(framer2)):
        expidr2=framer2[j][9]
        if expidr1==expidr2:
            #combine frameb1 and frameb2
            dump2h5(framer1[i],framer2[j],expidr1,hdf5output,'r')
            framer2.remove(framer2[j])
            hit=1
            break
    if hit==0:
        single_dump2h5(framer1[i],expidr1,hdf5output,'r')
for i in range(0,len(framer2)):
    expidr2=framer2[i][9]
    single_dump2h5(framer2[i],expidr2,hdf5output,'r')

In [1]:
'spFrame-b1' in 'spCFrame-b1-00135668.fits'

False

In [ ]:
id5290/55862/exposures/135671/b/calib
id5290/55862/exposures/135671/r/calib
